In [64]:
import Control.Applicative

In [3]:
newtype Identity a = Identity { runIdentity :: a } deriving (Eq, Show)

In [2]:
newtype Compose f g a = Compose { getCompose :: f (g a) } deriving (Eq, Show)

In [14]:
k = Compose [Just (1 :: Int), Nothing]
:t k

k :: Compose [] Maybe Int

In [15]:
instance Functor Identity where
    fmap f (Identity a) = Identity (f a)

In [16]:
instance (Functor f, Functor g) => Functor (Compose f g) where
    fmap f (Compose fga) = Compose $ (fmap . fmap) f fga

In [18]:
fmap (+1) k

Compose {getCompose = [Just 2,Nothing]}

In [89]:
{-# LANGUAGE InstanceSigs #-}
instance (Applicative f, Applicative g) => Applicative (Compose f g) where
    pure :: a -> Compose f g a
    pure = Compose . pure . pure
    
    (<*>) :: Compose f g (a -> b) -> Compose f g a -> Compose f g b
    (<*>) (Compose f') (Compose a') = Compose $ (fmap (<*>) f') <*> a'
    
{--
fmap :: (a -> b) -> f a -> f b
<*> :: f1 (a -> b) -> f1 a -> f1 b

fmap (<*>) :: f (f1 (a -> b)) -> f (f1 a -> f1 b)
f' ~ f g (a -> b)
f'' ~ fmap (<*>) f' :: f (g a -> g b)
a' ~ f g a

Therefore,
f'' <*> a' :: f g b
--}

Line 6: Redundant bracket
Found:
(fmap (<*>) f') <*> a'
Why not:
fmap (<*>) f' <*> a'

In [86]:
Compose [Just (+2), Just (*2)] <*> k

Compose {getCompose = [Just 3,Nothing,Just 2,Nothing]}

In [99]:
instance (Monad f, Monad g) => Monad (Compose f g) where
    return = pure
    
    (>>=) :: Compose f g a -> (a -> Compose f g b) -> Compose f g b
    -- (>>=) (Compose fga) f = fga >>= (\x -> x >>= (\y -> getCompose $ f y)) 
    (>>=) = undefined

{--
\x -> x :: ga (\y -> y :: a)
x >>= \y :: g (f g a)
y >>= (f :: Compose f g b) :: 
--}

In [97]:
instance (Foldable f, Foldable g) => Foldable (Compose f g) where
    foldMap f (Compose fga) = foldMap (foldMap f) fga

In [106]:
instance (Traversable f, Traversable g) => Traversable (Compose f g) where
    traverse :: Applicative f' => (a -> f' b) -> Compose f g a -> f' (Compose f g b) 
    traverse f (Compose fga) = fmap Compose (traverse (traverse f) fga)

<h4>BiFunctory Type Class</h4>

In [107]:
class BiFunctor p where
    {-# MINIMAL bimap | first, second #-}
    
    bimap :: (a -> b) -> (c -> d) -> p a c -> p b d
    bimap f g = first f . second g
    
    first :: (a -> b) -> p a c -> p b c
    first f = bimap f id
    
    second :: (c -> d) -> p a c -> p a d
    second = bimap id

In [110]:
data Deux a b = Deux a b deriving (Eq, Show)

instance BiFunctor Deux where
    bimap f g (Deux a b) = Deux (f a) (g b)

In [111]:
x = Deux 1.1 "rohit"
bimap (+1) (++ " gijare") x

Deux 2.1 "rohit gijare"

In [114]:
data Const a b = Const a deriving (Eq, Show)

instance BiFunctor Const where
    bimap f _ (Const a) = Const $ f a

In [115]:
y = Const 1.3
bimap (+1) (++ " gijare") y

Const 2.3

In [116]:
data Drei a b c = Drei a b c deriving (Eq, Show)

instance BiFunctor (Drei a) where
    bimap f g (Drei a b c) = Drei a (f b) (g c)

In [118]:
z = Drei 1.5 3.0 "Rohit"
bimap (+1) (++ " gijare") z

Drei 1.5 4.0 "Rohit gijare"

In [126]:
data SuperDrei a b c = SuperDrei a b deriving (Eq, Show)

instance BiFunctor (SuperDrei a) where
    bimap f _ (SuperDrei a b) = SuperDrei a (f b)

In [128]:
z' = SuperDrei "Rohit" "Gijare"
bimap (++ " GG ") (+1) z'

SuperDrei "Rohit" "Gijare GG "

In [130]:
data SemiDrei a b c = SemiDrei a

instance BiFunctor (SemiDrei a) where
    bimap _ _ (SemiDrei a) = SemiDrei a

In [133]:
data Quadriceps a b c d = Quadrazzz a b c d deriving (Eq, Show)

instance BiFunctor (Quadriceps a b) where
    bimap f g (Quadrazzz a b c d) = Quadrazzz a b (f c) (g d)

In [134]:
z'' = Quadrazzz 1 2 (Just 1.1) 2.2
bimap (fmap (+1)) (+2) z''

Quadrazzz 1 2 (Just 2.1) 4.2

In [138]:
data Sum a b = Lefty a | Righty b deriving (Eq, Show)

instance BiFunctor Sum where
    bimap f _ (Lefty a) = Lefty $ f a
    bimap _ g (Righty b) = Righty $ g b

In [139]:
x'  = Lefty "Rohit"
x'' = Righty "Exception"

bimap (++ " Gijare") (++ " Error") x'
bimap (++ " Gijare") (++ " Error") x''

Lefty "Rohit Gijare"

Righty "Exception Error"